# Task 7. Knolwedge inference (disease name) evaluation
- use [MASK] token

In [ ]:
from __future__ import absolute_import, division, print_function

import argparse
import csv
import json
import logging
import os
import random
import sys

import numpy as np
import torch
import torch.nn.functional as F

# import torch.multiprocessing as mp
# import torch.utils.data.distributed

from transformers import (WEIGHTS_NAME, AdamW, BertConfig,
                          BertForTokenClassification, BertTokenizer,
                          get_linear_schedule_with_warmup, 
                          BertPreTrainedModel, BertModel)

from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
# from transformers.models.bert.modeling_bert import Sentence_classifier


from torch import nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
# from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from seqeval.metrics import classification_report
from seqeval.metrics import sequence_labeling # 따로 결과를 뽑아내고 싶음

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)
import codecs

try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
    
import glob
import time

import easydict

import six
import tqdm
from tqdm import tqdm, trange
import collections

from seqeval.metrics import classification_report

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

import re

import shutil

In [ ]:
class TrainingInstance_ext(object):
    """A single training instance (sentence pair)."""
    def __init__(self, input_ids, input_mask, segment_ids, label_ids, mask_location):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        self.mask_location = mask_location


# Predict Mask

In [ ]:
class MaskEntityClassifier(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super().__init__(config)
        self.bert = BertModel(config, add_pooling_layer=False)
        self.mask_predict = nn.Linear(config.hidden_size, num_labels)
        self.init_weights()

    def forward(self, input_ids, label_ids, mask_location, 
                attention_mask=None, token_type_ids=None, 
                position_ids=None, head_mask=None, inputs_embeds=None,
                output_attentions=None, output_hidden_states=None,
                return_dict=None, split_num=1):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = outputs[0]
        label_ids = label_ids.view(label_ids.size()[0]*label_ids.size()[1])
        logits = self.mask_predict(sequence_output)
        pred_score = nn.functional.log_softmax(logits, dim=-1)
        pred_score = pred_score.detach().cpu()
        logits = logits.view(logits.size()[0]*logits.size()[1], logits.size()[2]) # 2560, 63
        loss_fct = CrossEntropyLoss(ignore_index=-1)
        loss = loss_fct(logits, label_ids)
        
        return loss, pred_score


## Evaluate

In [ ]:
def evaluate(args):
    ############################################################################
    ##              Multi-GPUs, Distributed settings
    ############################################################################ 
    if args.server_ip and args.server_port:
    # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd
        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()
    
    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        n_gpu = 1
        # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.distributed.init_process_group(backend='nccl')
    logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args.local_rank != -1), args.fp16))
    
    print("n_gpu: ", n_gpu)
    
    
    ############################################################################
    ##              batch size, gradient_accumulation_steps
    ############################################################################ 
    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            args.gradient_accumulation_steps))
    args.eval_batch_size = args.eval_batch_size // args.gradient_accumulation_steps
    print("args.eval_batch_size: ", args.eval_batch_size)
    
    ###############################################################
    #                Functions for Dataload (load cache data)
    ###############################################################
    # 파일 목록 획득
    def get_file_arrays(data_path):
        data_path = data_path.split(",")
        print("data_path: ", data_path)
        
        cache_files_all = []
        for i in range(len(data_path)):
            path = data_path[i].strip()
            files = glob.glob(path)
            cache_files_all = cache_files_all + files
        cache_files_all.sort()
        
        print("Got " + str(len(cache_files_all)) + " cache_files")
        
        return cache_files_all
    
    def read_files(cache_files_pieces):
        eval_features = []
        for c in range(len(cache_files_pieces)):
            with open(cache_files_pieces[c], 'rb') as input:
                eval_features = eval_features + pickle.load(input)
                
        num_eval_examples = len(eval_features)
        
        all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_ids for f in eval_features], dtype=torch.long)
        all_mask_location = torch.tensor([f.mask_location for f in eval_features], dtype=torch.long)
        
        all_input_ids = torch.squeeze(all_input_ids)
        all_input_mask = torch.squeeze(all_input_mask)
        all_segment_ids = torch.squeeze(all_segment_ids)
        all_label_ids = torch.squeeze(all_label_ids)
        all_mask_location = torch.squeeze(all_mask_location)
        
        eval_data = TensorDataset(
            all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_mask_location
        )
        
        if args.local_rank == -1:
            eval_sampler = SequentialSampler(eval_data)
        else:
            eval_sampler = DistributedSampler(eval_data)
        
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)
        
        return eval_dataloader, num_eval_examples
    
    def read_file(cache_file):
        eval_features = []
        with open(cache_file, 'rb') as input:
            eval_features = eval_features + pickle.load(input)
            
        num_eval_examples = len(eval_features)
        
        all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_ids for f in eval_features], dtype=torch.long)
        all_mask_location = torch.tensor([f.mask_location for f in eval_features], dtype=torch.long)
        
        if all_input_ids.size()[0]>1:
            all_input_ids = torch.squeeze(all_input_ids)
            all_input_mask = torch.squeeze(all_input_mask)
            all_segment_ids = torch.squeeze(all_segment_ids)
            all_label_ids = torch.squeeze(all_label_ids)
            all_mask_location = torch.squeeze(all_mask_location)
        
        eval_data = TensorDataset(
            all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_mask_location
        )
        
        if args.local_rank == -1:
            eval_sampler = SequentialSampler(eval_data)
        else:
            eval_sampler = DistributedSampler(eval_data)
        
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)
        
        return eval_dataloader, num_eval_examples
    
    ###############################################################
    #            Dataload (load cache data) - Train data
    ###############################################################
    cache_files = get_file_arrays(args.data_dir)
    print("cache_files: ", cache_files)
    
    ############################################################################
    ##                      labels
    ############################################################################
    def read_labels(label_path="./data/labels.txt"):
        label2idx = {}
        idx2label = {}
        
        file = open(label_path, "r")
        lines = file.readlines()
        file.close()

        for l in range(len(lines)):
            line = lines[l].strip()
            label2idx[line.replace(",", "/")] = l
            idx2label[l] = line.replace(",", "/")

        return label2idx, idx2label

    label2idx, idx2label = read_labels(label_path="./data/labels.txt")


    ############################################################################
    ##                      Prepare model
    ############################################################################
    print("loading weights from checkpoint (", args.checkpoint, ")")
    #config = BertConfig.from_pretrained(args.checkpoint, num_labels=num_labels)
    config = BertConfig.from_pretrained(args.checkpoint)
    model = MaskEntityClassifier.from_pretrained(args.checkpoint,
              from_tf = False,
              config = config, num_labels=len(label2idx))
    print("loaded weights from checkpoint (", args.checkpoint, ")")
    
    if args.local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab
    print("device: ", device)
    
    model.to(device)    
    
    ############################################################################
    ##              save model
    ############################################################################
    def save_model(args, global_step):
        if not os.path.exists(args.output_dir+"/"+str(global_step)):
            os.makedirs(args.output_dir+"/"+str(global_step))

        # save model
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        model_to_save.save_pretrained(args.output_dir+"/"+str(global_step))
    
    ############################################################################
    ##              Start training
    ############################################################################
    global_step = -1
    nb_tr_steps = 0
    tr_loss = 0
    
    args.save_file_limit
    min_train_loss = 1000000
    save_checkpoints = []
    save_loss = []
    
    
    model.eval()
    
    # metric
    examples_count = 0.0
    hits_at_1 = 0.0
    hits_at_3 = 0.0
    hits_at_10 = 0.0
    mrr = 0.0
    
    def hit_at_k(pred, k, label):
        values, rank_idx = torch.topk(pred, k)
        return torch.where(rank_idx==label, 1, 0).sum().item()
    
    # mean reciprocal rank (MRR)
    def calc_mrr(pred, label):
        argsort_idx = pred.argsort() 
        reverse_rank = (argsort_idx == label).nonzero()
        reverse_rank = reverse_rank[0].float().add(1.0)
        reverse_rank_value = (1.0 / reverse_rank).sum().item()
        return reverse_rank_value
        
    for c in range(len(cache_files)):
        filename = cache_files[c].split("/")[-1].split(".")[0]
        eval_dataloader, num_eval_examples = read_file(cache_files[c])
        
        # 출력 결과 수집 변수
        outtext = ["label\tpred rank\ttopk predictions"]
        for step, batch in enumerate(tqdm(eval_dataloader, desc="Iteration")):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids, mask_location = batch
            loss, pred_score = model(input_ids=input_ids, label_ids=label_ids, 
                                mask_location=mask_location, 
                                attention_mask=input_mask, token_type_ids=segment_ids,
                                position_ids=None, head_mask=None, inputs_embeds=None,
                                output_attentions=None, output_hidden_states=None,
                                return_dict=None)        
            
            input_ids  = input_ids.detach().cpu()
            input_mask = input_mask.detach().cpu()
            label_ids  = label_ids.detach().cpu()
            mask_location = mask_location.detach().tolist()
            
            for b in range(len(input_ids)):
                # find [MASK]
                mask_loc = 0
                for m in range(len(mask_location[b])):
                    if mask_location[b][m]==1:
                        mask_loc = m
                        break
                
                label = label_ids[b][mask_loc].item()
                
                # hits@10, hits@3, hits@1
                pred_log_softmax = pred_score[b][mask_loc]
                
                topk = 10
                values, rank_idx = torch.topk(pred_log_softmax, len(idx2label))
                pred = []
                for t in range(len(rank_idx)):
                    if t==topk:
                        break
                    pred.append(idx2label[rank_idx[t].item()])
                
                rank = (rank_idx == label).nonzero().item()
                
                # label\tpred rank\ttopk predictions
                preds_str = idx2label[label]+"\t"+str(rank)+"\t"+"\t".join(pred)
                outtext.append(preds_str)
                
                ### evluation
                hits_at_1 += hit_at_k(pred=pred_log_softmax, k=1, label=label)
                hits_at_3 += hit_at_k(pred=pred_log_softmax, k=3, label=label)
                hits_at_10 += hit_at_k(pred=pred_log_softmax, k=10, label=label)
                mrr += calc_mrr(pred=pred_log_softmax, label=label)
                examples_count += 1
                
        
        # loop within a file 
        test_mode = args.data_dir.split("/")[-2]
        
        modelname = args.checkpoint.split("/")[-2]
        outpath = "./data/scores/"+str(test_mode)+"/"+str(modelname)
            
        if not os.path.exists(outpath):
            os.makedirs(outpath)

        file = open(outpath+"/"+str(filename)+".txt", "w")
        file.write("\n".join(outtext))
        file.close()
    
    # output scores
    out_scores = []
    hits_at_1_score = hits_at_1 / examples_count * 100
    hits_at_3_score = hits_at_3 / examples_count * 100
    hits_at_10_score = hits_at_10 / examples_count * 100
    mrr_score = mrr / examples_count * 100
    
    out_scores.append("hits_at_1_score\thits_at_3_score\thits_at_10_score\tmrr_score")
    out_scores.append(str(hits_at_1_score)+"\t"+str(hits_at_3_score)+"\t"+str(hits_at_10_score)+"\t"+str(mrr_score))
    
    if not os.path.exists(outpath):
        os.makedirs(outpath)

    file = open(outpath+"/hit_score.txt", "w")
    file.write("\n".join(out_scores))
    file.close()
    

In [ ]:
def main(args):
    parser = argparse.ArgumentParser()
    
    print("args.output_dir: ", args.output_dir)
    
    # check directories
    if not args.do_train and not args.do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")
    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train:
        raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
            
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    
    if args.do_train:
        train(args)
    
    if args.do_eval:
        evaluate(args)

# finetune

In [ ]:
data_dirs = [
    './cache/bertbase_cased/test/*.cache',
    './cache/mbert_cased/test/*.cache',
    './cache/biobert/test/*.cache',
    './cache/kobert/test/*.cache',    
]

# pretrained models
checkpoints = [
    "./finetuned/ver9.1.4_521121_epoch2/927",
    "./finetuned/ver8.1.4_1142642_epoch2/927",
    './finetuned/ver11.1.4_521079_epoch2/927',
    './finetuned/ver12.1.4_407013_epoch2/927'
]

assert len(data_dirs)==len(checkpoints)

for d in range(len(data_dirs)):
    output_dir = "./finetuned/"+str(checkpoints[d].split("/")[-2])
    print("output_dir: ", output_dir)
                                   
    args = easydict.EasyDict({
        'data_dir':data_dirs[d],
        'eval_batch_size':5,
        'gradient_accumulation_steps':1, 
        'save_globalstep':1,
        'checkpoint':checkpoints[d],
        'output_dir':output_dir,
        'save_step':1,
        'num_files_on_memory':1,
        
        'do_train':False,
        'do_eval':True,
        'do_lower_case':True,
        'fp16_allreduce':False,
        'seed':42,
        'no_cuda':False,
        'use_adasum':False,
        'learning_rate':3e-05,
        'warmup_proportion':1.00,
        'num_train_epochs':2,
        'save_file_limit':1,        
        'weight_decay':0.01,
        'adam_epsilon':1e-8,
        'gradient_predivide_factor':1.0,
        'fp16':False,
        'max_grad_norm':1.0,
        'local_rank':-1,
        'fp16_opt_level':'O1',
        'loss_scale':0,
        
        'server_ip':None,
        'server_port':None,
        'cuda':True,
    })
    
    
    # configs
    if "mbert_cased" == str(data_dirs[d].split("/")[2]):
        print("multilingualbert configs")
        # multilingual bert config
        args["attention_probs_dropout_prob"] = 0.1
        args["directionality"] = "bidi"
        args["hidden_act"] = "gelu"
        args["hidden_dropout_prob"] = 0.1
        args["hidden_size"] = 768
        args["initializer_range"] = 0.02
        args["intermediate_size"] = 3072
        args["max_position_embeddings"] = 512
        args["num_attention_heads"] =  12
        args["num_hidden_layers"] = 12
        args["pooler_fc_size"] = 768
        args["pooler_num_attention_heads"] = 12
        args["pooler_num_fc_layers"] = 3
        args["pooler_size_per_head"] = 128
        args["pooler_type"] = "first_token_transform"
        args["type_vocab_size"] = 2
        args["vocab_size"] = 119547
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    elif "mbert_uncased" == str(data_dirs[d].split("/")[2]):
        print("multilingualbert configs")
        # multilingual bert config
        args["attention_probs_dropout_prob"] = 0.1
        args["directionality"] = "bidi"
        args["hidden_act"] = "gelu"
        args["hidden_dropout_prob"] = 0.1
        args["hidden_size"] = 768
        args["initializer_range"] = 0.02
        args["intermediate_size"] = 3072
        args["layer_norm_eps"] = 1e-12
        args["max_position_embeddings"] = 512
        args["model_type"] = "bert"
        args["num_attention_heads"] =  12
        args["num_hidden_layers"] = 12
        args["pad_token_id"] = 0
        args["pooler_fc_size"] = 768
        args["pooler_num_attention_heads"] = 12
        args["pooler_num_fc_layers"] = 3
        args["pooler_size_per_head"] = 128
        args["pooler_type"] = "first_token_transform"
        args["type_vocab_size"] = 2
        args["vocab_size"] = 105879
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    elif "bertbase_uncased" == str(data_dirs[d].split("/")[2]):
        args['vocab_size']=30522 # bert-base-uncased
        args['hidden_size']=768
        args['num_hidden_layers']=12
        args['num_attention_heads']=12
        args['hidden_act']='gelu'
        args['intermediate_size']=3072
        args['hidden_dropout_prob']=0.1
        args['attention_probs_dropout_prob']=0.1
        args['max_position_embeddings']=512
        args['type_vocab_size']=2
        args['initializer_range']=0.02
        args['layer_norm_eps']=1e-12
        args['gradient_checkpointing']=None
        args['position_embedding_type']=None
        args['use_cache']=None
        args['classifier_dropout']=None
        args["cls_id"] = 101
        args["sep_id"] = 102

    elif "bertbase_cased" == str(data_dirs[d].split("/")[2]):
        args['vocab_size']=28996 # bert-base-cased
        args['attention_probs_dropout_prob']=0.1
        args['hidden_act']='gelu'
        args['hidden_dropout_prob']=0.1
        args['hidden_size']=768
        args['initializer_range']=0.02
        args['intermediate_size']=3072
        args['layer_norm_eps']=1e-12
        args['max_position_embeddings']=512
        args['model_type']="bert"
        args['num_hidden_layers']=12
        args['num_attention_heads']=12
        args['pad_token_id']=0        
        args['type_vocab_size']=2
        args['gradient_checkpointing']=None
        args['position_embedding_type']=None
        args['use_cache']=None
        args['classifier_dropout']=None
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    # configs
    elif "biobert" == str(data_dirs[d].split("/")[2]):
        # BioBERT
        print("biobert configs")
        args["attention_probs_dropout_prob"]= 0.1
        args["hidden_act"] = "gelu"
        args["hidden_dropout_prob"] = 0.1
        args["hidden_size"] = 768
        args["initializer_range"] = 0.02
        args["intermediate_size"] = 3072
        args["max_position_embeddings"] = 512
        args["num_attention_heads"] = 12
        args["num_hidden_layers"] = 12
        args["type_vocab_size"] = 2
        args["vocab_size"] = 28996
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    elif "kobert" == str(data_dirs[d].split("/")[2]):
        print("kobert configs")
        args["attention_probs_dropout_prob"]= 0.1
        args["gradient_checkpointing"]= False
        args["hidden_act"]= "gelu"
        args["hidden_dropout_prob"]= 0.1
        args["hidden_size"]= 768
        args["initializer_range"]= 0.02
        args["intermediate_size"]= 3072
        args["layer_norm_eps"]= 1e-12
        args["max_position_embeddings"]= 512
        args["model_type"]= "bert"
        args["num_attention_heads"]= 12
        args["num_hidden_layers"]= 12
        args["pad_token_id"]= 1
        args["type_vocab_size"]= 2
        args["vocab_size"]= 8002
        args["author"]= "Heewon Jeon(madjakarta@gmail.com)"
        args["kobert_version"]= 1.0
        args["cls_id"] = 2
        args["sep_id"] = 3
                
    else:
        print("MY BERT")
        args['vocab_size']=30014
        args['hidden_size']=768
        args['num_hidden_layers']=12
        args['num_attention_heads']=12
        args['hidden_act']='gelu'
        args['intermediate_size']=3072
        args['hidden_dropout_prob']=0.1
        args['attention_probs_dropout_prob']=0.1
        args['max_position_embeddings']=512
        args['type_vocab_size']=2
        args['initializer_range']=0.02
        args['layer_norm_eps']=1e-12
        args['gradient_checkpointing']=None
        args['position_embedding_type']=None
        args['use_cache']=None
        args['classifier_dropout']=None
        args["cls_id"] = 4
        args["sep_id"] = 5
        
    main(args)

# score_gather

In [ ]:
import glob
def score_gather():
    path = "./data/scores/test/*"
    
    outtext = ["modelname\thits_at_1_score\thits_at_3_score\thits_at_10_score\tmrr_score"]
    
    modelnames = glob.glob(path)
    modelnames.sort()
    
    for m in range(len(modelnames)):
        modelname = modelnames[m].split("/")[-1]
        
        scorefile = modelnames[m]+"/hit_score.txt"
        file = open(scorefile, "r")
        lines = file.readlines()
        file.close()
        outtext.append(modelname+"\t"+lines[1].strip())
    
    file = open("./data/scores/scores_test.txt", "w")
    file.write("\n".join(outtext))
    file.close()
    
score_gather()
    